# Getting set up

## Setting up the notebook

In [1]:
### Installing dependencies
!pip install openai

!apt-get update
!apt-get install -y iverilog

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,242 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,96

## Use LLM to generate Verilog code from natural language description

### Example 1: binary_to_bcd

In [ ]:
! mkdir -p binary_to_bcd
! cd binary_to_bcd && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v

In [83]:
verilog_generation_prompt = '''
I am trying to create a Verilog model binary_to_bcd_converter for a binary to binary-coded-decimal converter. It must meet the following specifications:
	- Inputs:
		- Binary input (5-bits)
	- Outputs:
		- BCD (8-bits: 4-bits for the 10's place and 4-bits for the 1's place)

How would I write a design that meets these specifications?
'''

Now you can use the openai library to generate text from your prompt.

Before using LLMs for Verilog generation, you have to first get the api-key:

openai llms: https://platform.openai.com/api-keys
deepseek llms: https://platform.deepseek.com/sign_in
free llms provided by NVIDIA: https://build.nvidia.com/explore/discover

In [85]:
from openai import OpenAI
import os
client = OpenAI(
    api_key = ""
)

completion = client.chat.completions.create(
  model = "gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

To create a Verilog module for a binary to binary-coded decimal (BCD) converter, we need to process a 5-bit binary input and convert it into an 8-bit BCD output. The BCD representation will hold the tens place and the ones place, each using 4 bits.

The algorithm typically used for binary to BCD conversion is the double-dabble algorithm. This algorithm involves shifting the binary number and conditionally adding 3 to the BCD representation when necessary to ensure each BCD digit is valid (i.e., less than 10).

Here’s a simple Verilog implementation of a binary to BCD converter based on these specifications:

```verilog
module binary_to_bcd_converter (
    input wire [4:0] binary_input, // 5-bit binary input
    output reg [7:0] bcd_output     // 8-bit BCD output (4 bits for tens, 4 bits for ones)
);

// Internal registers for BCD representation
reg [3:0] bcd_tens; // Tens place
reg [3:0] bcd_ones; // Ones place
integer i; // Loop variable

always @* begin
    // Initialize BCD values t

Next, extract the generated verilog code from LLM response.

In [86]:
output_verilog_code = '''
module binary_to_bcd_converter (
    input wire [4:0] binary_input, // 5-bit binary input
    output reg [7:0] bcd_output     // 8-bit BCD output (4 bits for tens, 4 bits for ones)
);

// Internal registers for BCD representation
reg [3:0] bcd_tens; // Tens place
reg [3:0] bcd_ones; // Ones place
integer i; // Loop variable

always @* begin
    // Initialize BCD values to 0
    bcd_tens = 4'b0000;
    bcd_ones = 4'b0000;

    // Convert binary to BCD using double-dabble algorithm
    for (i = 0; i < 5; i = i + 1) begin
        // Shift left the BCD digits to make room for the next binary bit
        if (bcd_tens >= 5)
            bcd_tens = bcd_tens + 4'b0011; // Add 3 to tens place if necessary

        if (bcd_ones >= 5)
            bcd_ones = bcd_ones + 4'b0011; // Add 3 to ones place if necessary

        // Shift BCD values left to add a new binary bit
        {bcd_tens, bcd_ones} = {bcd_tens, bcd_ones} << 1;

        // Add the next bit of the binary input
        bcd_ones[0] = binary_input[4 - i]; // Add the next bit of the binary input
    end

    // Combine BCD places into the final output
    bcd_output = {bcd_tens, bcd_ones};
end

endmodule
'''
filename = "binary_to_bcd/binary_to_bcd.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

Use iverilog to verify the correctness of LLM generated verilog

In [87]:
!cd binary_to_bcd/ && iverilog -g2012 -o binary_to_bcd.vvp binary_to_bcd.v binary_to_bcd_tb.v && vvp binary_to_bcd.vvp

Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed!


### Example 2: Sequence detector

In [ ]:
! mkdir -p sequence_detector
! cd sequence_detector && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/sequence_detector_tb.v

In [63]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a sequence detector. It must meet the following specifications:
	- Inputs:
		- Clock
		- Active-low reset
		- Data (3 bits)
	- Outputs:
		- Sequence found

While enabled, it should detect the following sequence of binary input values:
	- 0b001
	- 0b101
	- 0b110
	- 0b000
	- 0b110
	- 0b110
	- 0b011
	- 0b101

How would I write a design that meets these specifications?
'''

In [64]:
from openai import OpenAI
import os
client = OpenAI(
    api_key = ""
)

completion = client.chat.completions.create(
  model = "gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

To create a sequence detector in Verilog that detects the specified sequences of 3-bit binary input values, you can use a finite state machine (FSM) approach. The sequences we need to detect are 0b001, 0b101, 0b110, 0b000, 0b110, 0b110, 0b011, and 0b101. 

Below is a sample implementation of a Verilog model for your sequence detector:

```verilog
module sequence_detector (
    input         clk,
    input         reset_n,
    input  [2:0] data,
    output reg    sequence_found
);

    // State encoding
    typedef enum reg [3:0] {
        S_IDLE      = 4'b0000,
        S_001       = 4'b0001,
        S_101       = 4'b0010,
        S_110       = 4'b0011,
        S_000       = 4'b0100,
        S_011       = 4'b0101
    } state_t;

    // State variables
    state_t current_state, next_state;

    // Sequence found flag
    reg found_flag;

    // State transition
    always @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            current_state <= S_IDLE;
        end

Next, extract the generated verilog code from LLM response.

In [65]:
output_verilog_code = '''
module sequence_detector (
    input         clk,
    input         reset_n,
    input  [2:0] data,
    output reg    sequence_found
);

    // State encoding
    typedef enum reg [3:0] {
        S_IDLE      = 4'b0000,
        S_001       = 4'b0001,
        S_101       = 4'b0010,
        S_110       = 4'b0011,
        S_000       = 4'b0100,
        S_011       = 4'b0101
    } state_t;

    // State variables
    state_t current_state, next_state;

    // Sequence found flag
    reg found_flag;

    // State transition
    always @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            current_state <= S_IDLE;
        end else begin
            current_state <= next_state;
        end
    end

    // Next state logic
    always @(*) begin
        case (current_state)
            S_IDLE: begin
                if (data == 3'b001)
                    next_state = S_001;
                else
                    next_state = S_IDLE;
            end
            S_001: begin
                if (data == 3'b101)
                    next_state = S_101;
                else
                    next_state = S_IDLE;
            end
            S_101: begin
                if (data == 3'b110)
                    next_state = S_110;
                else
                    next_state = S_IDLE;
            end
            S_110: begin
                if (data == 3'b000)
                    next_state = S_000;
                else
                    next_state = S_IDLE;
            end
            S_000: begin
                if (data == 3'b110)
                    next_state = S_110;
                else
                    next_state = S_IDLE;
            end
            // Return to S_IDLE for any unrecognized states
            default: begin
                next_state = S_IDLE;
            end
        endcase
    end

    // Output logic
    always @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            sequence_found <= 0;
        end else begin
            // Set found_flag only for successful transitions
            sequence_found <= (current_state == S_110 && next_state == S_011) ||
                              (current_state == S_011 && next_state == S_101);
        end
    end

endmodule
'''
filename = "sequence_detector/sequence_detector.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [66]:
!cd sequence_detector/ && iverilog -g2012 -o sequence_detector.vvp sequence_detector.v sequence_detector_tb.v && vvp sequence_detector.vvp

Error: Cycle 8, Expected: 1, Got: 0


### Example 3: Shift Register

In [ ]:
! mkdir -p shift_register
! cd shift_register && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/shift_register_tb.v

In [17]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a shift register. It must meet the following specifications:
	- Inputs:
		- Clock
		- Active-low reset
		- Data (1 bit)
		- Shift enable
	- Outputs:
		- Data (8 bits)

How would I write a design that meets these specifications?
'''

In [18]:
from openai import OpenAI

client = OpenAI(
  api_key = ""
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)



To create a Verilog model for an 8-bit shift register with the specified inputs and outputs, you'll need to define the behavior based on the clock, reset, and control signals. Below is an example of how you might write this design:

```verilog
module shift_register (
  input wire clk,        // Clock signal
  input wire reset_n,    // Active-low reset signal
  input wire data_in,    // Data input
  input wire shift_enable,
  output reg [7:0] data_out
);

always @(posedge clk or negedge reset_n) begin
  if (!reset_n) begin
    // Active-low reset, so when reset_n is 0, reset data_out to 0
    data_out <= 8'b00000000;
  end else if (shift_enable) begin
    // Shift data on the rising edge of the clock when shift_enable is 1
    data_out <= {data_out[6:0], data_in}; 
  end
end

endmodule
```

### Explanation:
1. **Module Declaration**:
   - The module is named `shift_register`.
   - The module takes four input signals (`clk`, `reset_n`, `data_in`, `shift_enable`) and one output signal (`d

In [19]:
output_verilog_code = '''
module shift_register (
  input wire clk,        // Clock signal
  input wire reset_n,    // Active-low reset signal
  input wire data_in,    // Data input
  input wire shift_enable,
  output reg [7:0] data_out
);

always @(posedge clk or negedge reset_n) begin
  if (!reset_n) begin
    // Active-low reset, so when reset_n is 0, reset data_out to 0
    data_out <= 8'b00000000;
  end else if (shift_enable) begin
    // Shift data on the rising edge of the clock when shift_enable is 1
    data_out <= {data_out[6:0], data_in};
  end
end

endmodule
'''
filename = "shift_register/shift_register.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [25]:
!cd sequence_detector/ && iverilog -o sequence_detector.vvp sequence_detector.v sequence_detector_tb.v && vvp sequence_detector.vvp

sequence_detector.v:10: syntax error
sequence_detector.v:10: error: Invalid module instantiation
sequence_detector.v:14: syntax error
sequence_detector.v:14: error: Invalid module instantiation


### Example 4: Abro State Machine

In [ ]:
! mkdir -p abro_state_machine
!cd abro_state_machine && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/abro_state_machine_tb.v

In [ ]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for an ABRO state machine. It must meet the following specifications:
    - Inputs:
        - Clock
        - Active-low reset
        - A
        - B
    - Outputs:
        - O
        - State

Other than the main output from ABRO machine, it should output the current state of the machine for use in verification.

The states for this state machine should be one-hot encoded.

How would I write a design that meets these specifications?
'''

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key = ""
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

In [ ]:
output_verilog_code = '''
'''
filename = "abro_state_machine/abro_state_machine.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [20]:
!cd abro_state_machine/ && iverilog -o abro_state_machine.vvp abro_state_machine.v abro_state_machine_tb.v && vvp abro_state_machine.vvp

Error: Test case 0 failed. Expected: 00001010, Got: xxxxxxxx


### Example 5: Dice Roller

In [3]:
!mkdir -p dice_roller
!cd dice_roller && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/dice_roller_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1367  100  1367    0     0   7500      0 --:--:-- --:--:-- --:--:--  7510


In [4]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a simulated dice roller. It must meet the following specifications:
    - Inputs:
        - Clock
        - Active-low reset
        - Die select (2-bits)
        - Roll
    - Outputs:
        - Rolled number (up to 8-bits)

The design should simulate rolling either a 4-sided, 6-sided, 8-sided, or 20-sided die, based on the input die select. It should roll when the roll input goes high and output the random number based on the number of sides of the selected die.

How would I write a design that meets these specifications?
'''

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2114  100  2114    0     0  12358      0 --:--:-- --:--:-- --:--:-- 12435


In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key = ""
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

In [ ]:
output_verilog_code = '''
'''
filename = "dice_roller/dice_roller.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [ ]:
!cd dice_roller/ && iverilog -o dice_roller.vvp dice_roller.v dice_roller_tb.v && vvp dice_roller.vvp

### Example 6: LFSR

In [ ]:
!mkdir -p lfsr
!cd lfsr && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/lfsr_tb.v

In [ ]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for an LFSR. It must meet the following specifications:
	- Inputs:
		- Clock
        - Active-low reset
	- Outputs:
		- Data (8-bits)

The initial state should be 10001010, and the taps should be at locations 1, 4, 6, and 7.

How would I write a design that meets these specifications?
'''

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key = ""
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

In [ ]:
output_verilog_code = '''
'''
filename = "lfsr/lfsr.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [ ]:
!cd lfsr/ && iverilog -o lfsr.vvp lfsr.v lfsr_tb.v && vvp lfsr.vvp

### Example 7: Sequence Generator

In [88]:
!mkdir -p sequence_generator
!cd sequence_generator && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/sequence_generator_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2344  100  2344    0     0   8897      0 --:--:-- --:--:-- --:--:--  8912


In [ ]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a sequence generator. It must meet the following specifications:
	- Inputs:
		- Clock
		- Active-low reset
		- Enable
	- Outputs:
		- Data (8 bits)

While enabled, it should generate an output sequence of the following hexadecimal values and then repeat:
	- 0xAF
	- 0xBC
	- 0xE2
	- 0x78
	- 0xFF
	- 0xE2
	- 0x0B
	- 0x8D

How would I write a design that meets these specifications?
'''

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key = ""
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

In [ ]:
output_verilog_code = '''
'''
filename = "sequence_generator/sequence_generator.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [ ]:
!cd sequence_generator/ && iverilog -o sequence_generator.vvp sequence_generator.v sequence_generator_tb.v && vvp sequence_generator.vvp

### Example 8: Traffic Light

In [ ]:
!mkdir -p traffic_light/
!cd traffic_light/ && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/traffic_light_tb.v

In [ ]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a traffic light state machine. It must meet the following specifications:
    - Inputs:
        - Clock
        - Active-low reset
        - Enable
    - Outputs:
        - Red
        - Yellow
        - Green

The state machine should reset to a red light, change from red to green after 32 clock cycles, change from green to yellow after 20 clock cycles, and then change from yellow to red after 7 clock cycles.

How would I write a design that meets these specifications?
'''

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key = ""
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1024, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

In [ ]:
output_verilog_code = '''
'''
filename = "traffic_light/traffic_light.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [ ]:
!cd traffic_light/ && iverilog -o traffic_light.vvp traffic_light.v traffic_light_tb.v && vvp traffic_light.vvp